In [1]:
import numpy as np
import keras as k
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [12]:
"""
This function reads all of the files and lables from the json file then opens those files and converts the
images to np arrays to be fed into the nn.
NOTE: this is not a good thing to do if your dataset is too big, this may crash your computer
"""
def imagegen():
    f = open('InfraredSolarModules/module_metadata.json',) 
    data = json.load(f) 
    f.close() 
    img = []
    cat = []
    for i in range(20000):
        if data[str(i)]['anomaly_class'] == 'No-Anomaly':
            pass
        else:
            img.append(np.array(k.preprocessing.image.load_img('InfraredSolarModules/' + data[str(i)]['image_filepath'], color_mode = "grayscale")))
            cat.append(data[str(i)]['anomaly_class'])
    return np.array(img), np.array(cat)

In [13]:
f = open('InfraredSolarModules/module_metadata.json',) 
data = json.load(f) 
f.close() 
print(data['15000'])

{'image_filepath': 'images/15000.jpg', 'anomaly_class': 'No-Anomaly'}


In [14]:
img, labels = imagegen() # load the data and labels
img = img.reshape(10000, 40, 24, 1) # reshape to show color channels
img = img / 255 # scale data

In [15]:
X_train, X_test, y_train, y_test = train_test_split(img, labels, test_size=0.2)
labelencoder = LabelEncoder() # Make a label encoder 
y_train = labelencoder.fit_transform(y_train) # encode labels to numbers
y_test = labelencoder.transform(y_test)
y_train = k.utils.to_categorical(y_train) # encode nubers to catagorical labels
y_test = k.utils.to_categorical(y_test)

In [16]:
X_train.shape

(8000, 40, 24, 1)

In [17]:
y_train.shape

(8000, 11)

In [26]:
# TODO: Upsample anomalies that don't happen often
counts = {}
for i in y_train:
    try:
        counts[str(i)] += 1
    except:
        counts[str(i)] = 0

In [27]:
counts

{'[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]': 13049,
 '[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]': 7409,
 '[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]': 9573,
 '[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]': 15039,
 '[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]': 10169,
 '[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]': 8649,
 '[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]': 12009,
 '[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]': 15101,
 '[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]': 6769,
 '[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]': 14005,
 '[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]': 11552}

In [25]:
for i in range(len(y_train)):
    if counts[str(y_train[i])] < 2000:
        new_X_train = np.array([np.fliplr(X_train[i]),
                                np.flipud(X_train[i]), 
                                np.flipud(np.fliplr(X_train[i])), 
                                X_train[i]*0.50,
                                X_train[i]*0.75,
                                np.fliplr(X_train[i])*0.75,
                                np.flipud(X_train[i])*0.75, 
                                np.flipud(np.fliplr(X_train[i]))*0.75,
                                X_train[i]+np.random.normal(0, .1, X_train[i].shape)])
        new_y_train = np.array([y_train[i], y_train[i], y_train[i], y_train[i], y_train[i], y_train[i], y_train[i], y_train[i], y_train[i]])
        X_train = np.concatenate((X_train, new_X_train), axis=0)
        y_train = np.concatenate((y_train, new_y_train), axis=0)



KeyboardInterrupt: 

In [20]:
X_train.shape

(66464, 40, 24, 1)

In [21]:
y_train.shape

(66464, 11)

In [28]:
model = k.Sequential()
model.add(k.layers.Conv2D(filters=15, kernel_size=(3,3), strides=(1, 1), padding="valid", input_shape=(40, 24, 1)))
model.add(k.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding="same"))
model.add(k.layers.Conv2D(filters=15, kernel_size=(3,3), strides=(1, 1), padding="valid"))
model.add(k.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding="same"))
model.add(k.layers.Flatten())
model.add(k.layers.Dense(675))
model.add(k.layers.Activation('relu'))
model.add(k.layers.Dense(11))
model.add(k.layers.Activation('softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 38, 22, 15)        150       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 19, 11, 15)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 17, 9, 15)         2040      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 9, 5, 15)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 675)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 675)               456300    
_________________________________________________________________
activation_2 (Activation)    (None, 675)              

In [29]:
for i in range(20):
    print(i)
    model.fit(X_train, y_train, epochs=1, batch_size=20, verbose=1, shuffle=True)
    model.evaluate(X_test, y_test, verbose=1)

0
63/63 [==============================] - 0s 4ms/step - loss: 1.3227 - accuracy: 0.5680
1
63/63 [==============================] - 0s 4ms/step - loss: 1.2893 - accuracy: 0.5850
2
63/63 [==============================] - 0s 4ms/step - loss: 1.4138 - accuracy: 0.5975
3
63/63 [==============================] - 0s 4ms/step - loss: 1.5673 - accuracy: 0.6105
4
63/63 [==============================] - 0s 4ms/step - loss: 1.7899 - accuracy: 0.6145
5
63/63 [==============================] - 0s 4ms/step - loss: 1.8794 - accuracy: 0.6170
6
63/63 [==============================] - 0s 4ms/step - loss: 2.0208 - accuracy: 0.6095
7
63/63 [==============================] - 0s 4ms/step - loss: 2.2283 - accuracy: 0.6255
8
63/63 [==============================] - 0s 4ms/step - loss: 2.3922 - accuracy: 0.6170
9
63/63 [==============================] - 0s 4ms/step - loss: 2.6862 - accuracy: 0.6035
10
63/63 [==============================] - 0s 4ms/step - loss: 2.9103 - accuracy: 0.6275
11
63/63 [==========

In [50]:
loss, acc = model.evaluate(X_test, y_test, verbose=1)
print(f"Prediction loss, accuracy: {loss, acc}")

188/188 [==============================] - 1s 7ms/step - loss: 2.1798 - accuracy: 0.6997
Prediction loss, accuracy: (2.179806709289551, 0.6996666789054871)


In [8]:
ls = np.array("118.0 117.0 114.0 117.0 123.0 131.0 136.0 141.0 146.0 150.0 153.0 153.0 151.0 150.0 150.0 153.0 156.0 159.0 157.0 157.0 160.0 159.0 156.0 156.0 156.0 156.0 155.0 152.0 152.0 152.0 154.0 152.0 148.0 147.0 146.0 146.0 141.0 135.0 135.0 137.0 112.0 112.0 112.0 112.0 116.0 126.0 137.0 146.0 147.0 145.0 141.0 143.0 147.0 148.0 149.0 150.0 150.0 150.0 148.0 147.0 148.0 147.0 144.0 144.0 144.0 144.0 144.0 145.0 146.0 147.0 148.0 147.0 144.0 140.0 138.0 136.0 133.0 130.0 126.0 124.0 112.0 112.0 113.0 116.0 120.0 127.0 135.0 141.0 142.0 141.0 141.0 142.0 144.0 148.0 151.0 152.0 151.0 150.0 148.0 147.0 146.0 145.0 144.0 142.0 141.0 141.0 141.0 141.0 140.0 139.0 138.0 139.0 141.0 140.0 137.0 134.0 132.0 128.0 121.0 117.0 109.0 110.0 112.0 118.0 126.0 132.0 136.0 138.0 140.0 142.0 143.0 141.0 140.0 145.0 148.0 150.0 149.0 147.0 146.0 145.0 142.0 141.0 140.0 137.0 136.0 137.0 138.0 139.0 135.0 132.0 130.0 130.0 131.0 131.0 131.0 128.0 124.0 118.0 115.0 113.0 99.0 100.0 103.0 111.0 121.0 130.0 137.0 142.0 142.0 139.0 135.0 132.0 130.0 134.0 137.0 139.0 139.0 138.0 139.0 138.0 135.0 134.0 132.0 128.0 124.0 124.0 124.0 125.0 126.0 127.0 129.0 128.0 126.0 125.0 124.0 122.0 117.0 111.0 110.0 111.0 90.0 91.0 93.0 98.0 106.0 116.0 127.0 134.0 135.0 131.0 125.0 127.0 132.0 132.0 131.0 132.0 131.0 130.0 130.0 131.0 131.0 130.0 128.0 126.0 124.0 123.0 121.0 121.0 123.0 127.0 129.0 131.0 132.0 128.0 122.0 117.0 115.0 114.0 112.0 111.0 88.0 91.0 95.0 99.0 103.0 109.0 119.0 128.0 130.0 129.0 124.0 125.0 129.0 130.0 130.0 130.0 129.0 128.0 127.0 127.0 128.0 128.0 128.0 126.0 125.0 124.0 124.0 124.0 124.0 124.0 126.0 125.0 124.0 119.0 115.0 113.0 113.0 114.0 109.0 106.0 86.0 88.0 93.0 101.0 109.0 117.0 124.0 129.0 129.0 128.0 125.0 126.0 129.0 129.0 129.0 129.0 128.0 126.0 126.0 126.0 128.0 128.0 126.0 124.0 121.0 120.0 119.0 118.0 118.0 118.0 117.0 116.0 114.0 111.0 109.0 110.0 111.0 112.0 107.0 103.0 77.0 81.0 88.0 97.0 106.0 115.0 124.0 132.0 135.0 134.0 130.0 130.0 131.0 129.0 128.0 129.0 129.0 128.0 129.0 129.0 130.0 130.0 129.0 128.0 127.0 127.0 127.0 127.0 129.0 129.0 126.0 124.0 123.0 123.0 122.0 116.0 110.0 104.0 96.0 92.0 55.0 55.0 57.0 71.0 90.0 111.0 125.0 134.0 138.0 138.0 136.0 136.0 136.0 136.0 136.0 136.0 135.0 134.0 135.0 137.0 141.0 143.0 145.0 144.0 143.0 141.0 139.0 136.0 137.0 137.0 133.0 132.0 131.0 130.0 126.0 115.0 100.0 84.0 71.0 64.0 64.0 66.0 71.0 84.0 100.0 117.0 129.0 136.0 139.0 139.0 138.0 140.0 144.0 146.0 147.0 147.0 146.0 146.0 144.0 144.0 146.0 148.0 151.0 150.0 147.0 144.0 141.0 138.0 136.0 135.0 133.0 133.0 132.0 132.0 131.0 126.0 114.0 100.0 90.0 84.0 81.0 84.0 90.0 101.0 111.0 119.0 124.0 126.0 128.0 130.0 131.0 132.0 134.0 135.0 135.0 135.0 134.0 133.0 131.0 130.0 132.0 133.0 134.0 134.0 133.0 131.0 128.0 126.0 125.0 124.0 125.0 126.0 126.0 126.0 125.0 121.0 114.0 105.0 100.0 99.0 78.0 77.0 78.0 89.0 100.0 110.0 115.0 116.0 117.0 117.0 119.0 120.0 122.0 124.0 124.0 123.0 120.0 117.0 115.0 114.0 115.0 116.0 118.0 119.0 119.0 119.0 118.0 117.0 115.0 114.0 113.0 113.0 113.0 113.0 113.0 110.0 103.0 94.0 90.0 88.0 45.0 46.0 51.0 67.0 85.0 102.0 112.0 117.0 120.0 121.0 121.0 120.0 118.0 120.0 121.0 122.0 122.0 122.0 121.0 120.0 120.0 120.0 121.0 123.0 124.0 125.0 124.0 122.0 121.0 119.0 117.0 115.0 112.0 109.0 104.0 97.0 83.0 68.0 58.0 53.0 58.0 61.0 68.0 82.0 96.0 109.0 117.0 122.0 128.0 132.0 133.0 130.0 124.0 123.0 124.0 126.0 127.0 128.0 128.0 128.0 126.0 124.0 124.0 125.0 126.0 128.0 127.0 126.0 125.0 125.0 124.0 121.0 118.0 116.0 111.0 104.0 95.0 85.0 77.0 74.0 83.0 86.0 92.0 100.0 109.0 116.0 121.0 124.0 130.0 133.0 134.0 133.0 129.0 127.0 126.0 125.0 126.0 127.0 129.0 130.0 129.0 128.0 126.0 124.0 123.0 125.0 125.0 124.0 123.0 122.0 120.0 119.0 118.0 117.0 115.0 112.0 108.0 103.0 97.0 95.0 82.0 87.0 95.0 102.0 107.0 109.0 113.0 115.0 118.0 120.0 121.0 124.0 126.0 126.0 125.0 124.0 125.0 125.0 125.0 125.0 125.0 125.0 124.0 123.0 125.0 126.0 126.0 124.0 123.0 121.0 119.0 117.0 114.0 111.0 108.0 104.0 101.0 98.0 95.0 94.0 78.0 82.0 89.0 94.0 99.0 102.0 108.0 112.0 115.0 116.0 117.0 119.0 121.0 121.0 122.0 125.0 126.0 125.0 122.0 120.0 120.0 121.0 125.0 127.0 130.0 130.0 129.0 126.0 124.0 121.0 118.0 116.0 113.0 110.0 107.0 103.0 98.0 93.0 90.0 88.0 76.0 79.0 84.0 89.0 93.0 98.0 102.0 106.0 109.0 112.0 114.0 115.0 116.0 117.0 118.0 120.0 120.0 120.0 119.0 117.0 116.0 118.0 120.0 122.0 123.0 124.0 121.0 117.0 116.0 114.0 112.0 111.0 110.0 109.0 105.0 99.0 93.0 87.0 82.0 79.0 71.0 72.0 76.0 84.0 93.0 100.0 103.0 104.0 107.0 110.0 113.0 115.0 115.0 115.0 115.0 114.0 113.0 113.0 115.0 115.0 112.0 112.0 114.0 115.0 116.0 117.0 116.0 114.0 113.0 112.0 110.0 108.0 105.0 103.0 99.0 95.0 90.0 86.0 83.0 81.0 68.0 70.0 74.0 83.0 91.0 97.0 100.0 99.0 100.0 103.0 107.0 109.0 110.0 109.0 108.0 106.0 106.0 107.0 109.0 108.0 104.0 102.0 102.0 103.0 104.0 106.0 107.0 107.0 104.0 102.0 101.0 100.0 98.0 92.0 89.0 90.0 88.0 85.0 76.0 70.0 61.0 63.0 67.0 73.0 80.0 86.0 90.0 92.0 93.0 95.0 97.0 96.0 96.0 96.0 96.0 95.0 96.0 97.0 99.0 99.0 97.0 95.0 92.0 91.0 90.0 92.0 92.0 92.0 91.0 90.0 91.0 90.0 88.0 84.0 80.0 78.0 75.0 71.0 65.0 61.0 52.0 57.0 63.0 71.0 77.0 81.0 84.0 85.0 85.0 84.0 84.0 84.0 84.0 83.0 82.0 83.0 84.0 85.0 86.0 87.0 89.0 88.0 85.0 82.0 80.0 79.0 80.0 82.0 82.0 82.0 81.0 80.0 77.0 75.0 71.0 67.0 64.0 62.0 61.0 61.0 50.0 53.0 59.0 65.0 70.0 73.0 76.0 79.0 80.0 79.0 79.0 79.0 79.0 79.0 78.0 80.0 81.0 81.0 78.0 79.0 83.0 83.0 80.0 79.0 78.0 76.0 76.0 76.0 74.0 74.0 75.0 75.0 72.0 67.0 63.0 63.0 62.0 60.0 53.0 50.0".split(' '))



In [11]:
ls = ls.reshape((40,24, 1))

In [18]:
vfunc = np.vectorize(lambda x : float(x) )

In [19]:
ls = vfunc(ls)

In [20]:
ls

array([[[118.],
        [117.],
        [114.],
        [117.],
        [123.],
        [131.],
        [136.],
        [141.],
        [146.],
        [150.],
        [153.],
        [153.],
        [151.],
        [150.],
        [150.],
        [153.],
        [156.],
        [159.],
        [157.],
        [157.],
        [160.],
        [159.],
        [156.],
        [156.]],

       [[156.],
        [156.],
        [155.],
        [152.],
        [152.],
        [152.],
        [154.],
        [152.],
        [148.],
        [147.],
        [146.],
        [146.],
        [141.],
        [135.],
        [135.],
        [137.],
        [112.],
        [112.],
        [112.],
        [112.],
        [116.],
        [126.],
        [137.],
        [146.]],

       [[147.],
        [145.],
        [141.],
        [143.],
        [147.],
        [148.],
        [149.],
        [150.],
        [150.],
        [150.],
        [148.],
        [147.],
        [148.],
        [147.],
    

In [15]:
model.save("model.h5")

In [ ]:
model = load_model('model.h5')